# Neural Nets and Multi-layer Perceptrons

In this notebook we will learn how we could use a neural network to predict cancer based on medical images. 

#### Start by importing packages

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons, load_breast_cancer
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


## Load the data

Again we will use a dataset built-in to Sklearn that includes data related to diagnosing breast cancer.

In [ ]:

cancer = load_breast_cancer()

# print('Dataset raw object', cancer)
print('Dataset description', cancer['DESCR'])


## Split into our training and testing XY sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

## Exercise 5.1: Scale the data

The Multilayer Perceptron (MLP) approach is one of the few that doesn't automatically scale the data, so let's do that. Here we will use Numpy to do it manually, though there are alternative built-in methods within scikit-learn.

In the code block below, use `X_train.mean(axis=0)` and similar functions to scale ALL of the X variables so that they have mean 0 and standard deviation 1. HINT: X_train and others are numpy arrays and so you can use fast raster math, e.g., `X_train - mean_on_train`.

In [ ]:
# Exercies 5.1 workspace



## Create the MLP model object and fit it

Using this new scaled training data, we are ready to define a Neural Net, Known here as a Multi-Layer-Perceptron (MLP) classifier. Because this next line hides away millions of other lines of code, you may want to explore it. In VS Code, you can navigate to a function's definition by placing your cursor in the function and press f-12. Try it in the cell below on the `MLPClassifier` code! The best documentation is often the code itself.


In [ ]:
mlp = MLPClassifier(random_state=0)

# Now fit it with the scaled X and y TRAINING data.
mlp.fit(X_train_scaled, y_train)

print(mlp)


## Assess the fit

Now we assess MLP's accuracy on the TRAINING and the TESTING data.

Notice here also I'm introducing another convenient way of combining strings and numbers. The {:.2f} specifies a placeholder for a 2-digit representation of a floating point number. The Format method then places that floating point value into that placeholder.


In [ ]:
score_train = mlp.score(X_train_scaled, y_train)
score_test = mlp.score(X_test_scaled, y_test)

print("Accuracy on training set: {:.3f}".format(score_train))
print("Accuracy on test set: {:.3f}".format(mlp.score(X_test_scaled, y_test)))


## Plot the inputs and hidden layers of the neural net

It can be hard perhaps to visualize what exaclty the neural net looks like (there is no coefficients table to simply look at). But here, it is small enough to actually visualize the coefficients within the network.

Below, we plot the coeffs_ array to see it.

In [ ]:

plt.figure(figsize=(20, 5))
plt.imshow(mlp.coefs_[0], interpolation='none', cmap='viridis')
plt.yticks(range(30), cancer.feature_names)
plt.xlabel("Columns in weight matrix")
plt.ylabel("Input feature")
plt.colorbar()
plt.show()

## Exercies 5.1.2: Understanding which features matter most

One of the massive challenges in Neural Nets is understanding why exactly it makes the predictions it does. Can you identify which input feature shows the largest positive effect on on cancer diagnosis?

You probably can't make heads or tails of it. Let's create a greatly simplified version of our neural network to try to see if we can understand it.

Specifically create a new `MLPClassifier` but this time make it have only a single hidden layer. Hint: use f-12 on the MLPClassifier code to see it's documentation and figure out what new input variable you sohuld specify when calling `mlp = MLPClassifier( .......  )`. Plot the output coefficients just like above. With only a single layer, the variables become somewhat more interpretable. 

Which variable now seems to have the largest positive impact?

In [ ]:
# Excercise 5.1.2 workspace

